In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pacswg

In [ ]:
# TODO: measurements are once every 30 seconds, convert them to time

In [ ]:
# service_name = 'bentoml-keras-toxic-comments'
# exp_name = 'res-2021-06-28_22-09-08'
# slo_timeout = 500
# trace_name = 'trace2'

# service_name = 'bentoml-pytorch-fashion-mnist'
# trace_name = 'trace_trace_wc'
# # exp_name = 'res-2021-07-29_18-18-28_proxy_no_controller'
# exp_name = 'res-2021-07-29_23-08-36_proxy'
# slo_timeout = 1000

service_name = 'bentoml-iris'
trace_name = 'trace_trace_wc'
# experiments
# exp_name = 'res-2021-08-03_11-46-33_proxy'
# exp_name = 'res-2021-08-03_16-39-31_proxy_no_controller'
# exp_name = 'res-2021-08-03_19-10-32_proxy'
# exp_name = 'res-2021-08-04_11-35-40_proxy'
# slo_timeout = 500
# exp_name = 'res-2021-08-04_15-54-03_proxy'
exp_name = 'res-2021-08-04_18-12-27_proxy'
slo_timeout = 200

exp_folder = f'results/{trace_name}/{service_name}'
exp_proxy_filename = f'{exp_folder}/{exp_name}.csv'
exp_reqs_filename = exp_proxy_filename.replace('proxy', 'reqs')

In [ ]:
df_proxy_stats = pd.read_csv(exp_proxy_filename, index_col=0, parse_dates=True)
df_res = pd.read_csv(exp_reqs_filename, index_col=0, parse_dates=True)

In [ ]:
plt.plot(df_proxy_stats['averageMaxBufferSize'])
plt.plot(df_proxy_stats['averageActualBatchSize'])

In [ ]:
plt.plot(df_proxy_stats['averageTimeoutRatio'])

In [ ]:
plt.plot(df_proxy_stats['reponseTimeAverage'])
plt.plot(df_proxy_stats['reponseTimeP50'])
plt.plot(df_proxy_stats['reponseTimeP95'])

In [ ]:
# plt.plot(df_proxy_stats['reponseTimeAverage'][5:30])
# plt.plot(df_proxy_stats['reponseTimeP50'][5:30])
# plt.plot(df_proxy_stats['reponseTimeP95'][5:30])

In [ ]:
plt.plot(df_proxy_stats['currentReplicaCount'])
plt.plot(df_proxy_stats['currentReadyReplicaCount'], ls='--')
print(df_proxy_stats['currentReadyReplicaCount'].mean())
print(df_proxy_stats['currentReadyReplicaCount'].max())

In [ ]:
plt.plot(df_proxy_stats['averageArrivalRate'])

## Other Useful Plots

In [ ]:
df_res['received_at'] = pd.to_datetime(df_res['received_at'])
df_res['response_at'] = pd.to_datetime(df_res['response_at'])

In [ ]:
# get a temporal data of means
temp_df = df_res.set_index('received_at').resample('T').mean()
temp_df.head()

In [ ]:

plt.plot(temp_df['upstream_response_time'])

In [ ]:
plt.plot(temp_df['upstream_request_count'])

In [ ]:
plt.plot(temp_df['queue_time_ms'], label='Queue Time (ms)')
plt.plot(temp_df['response_time_ms_server'], label='Response Time (ms)')
plt.plot(temp_df['upstream_response_time'], label='Upstream RT (ms)')

plt.legend()

In [ ]:
df_resample = df_res.set_index('received_at').resample('T')
p95_over_time = df_resample['response_time_ms_server'].quantile(0.95)
plt.plot(p95_over_time)

In [ ]:
# SLO miss rate
slo_miss_count = df_resample['response_time_ms_server'].apply(lambda x: np.sum(x > slo_timeout))
resampled_request_count = df_resample['response_time_ms_server'].apply(lambda x: len(x))
slo_miss_rates = slo_miss_count / resampled_request_count * 100
plt.plot(slo_miss_rates)
plt.ylabel('SLO Miss Rate (%)')
plt.xlabel('Time')

# Extract Experiment Parameters

In [ ]:
slo_miss_percentage = np.sum(df_res['response_time_ms_server'] > slo_timeout) / df_res.shape[0] * 100
print(f'slo_miss_percentage {slo_miss_percentage:4.3f}%')